In [ ]:
import numpy as np
from matplotlib import pyplot

In [ ]:
def f(x):
    return np.power(2., x)-1.

In [ ]:
x = np.linspace(0., 1., 100, dtype=np.float32)

In [ ]:
y = f(x)

In [ ]:
poly = np.polyfit(x, y, deg=2).astype(np.float32)

In [ ]:
fit = np.polyval(poly, x.astype(np.float32))

In [ ]:
pyplot.plot(x, y)
pyplot.plot(x, fit, color='r')

In [ ]:
poly

In [ ]:
np.finfo(np.float32)

In [ ]:
x = np.float32(0.)
print (bin(x.view(dtype=np.uint32).copy()))
exponent = ((x.view(dtype=np.uint32).copy() >> 23) & (255)) - 127
exponent

In [ ]:
np.log2(np.e)

In [ ]:
def exp_approx(x):
        # http://spfrnd.de/posts/2018-03-10-fast-exponential.html
        x = x * np.log2(np.e)
        xi = np.int32(np.floor(x))
        xf = x - xi
        y = np.power(np.float32(2.), xi)
        corr = np.float32(1.) + np.polyval(poly, xf).astype(np.float32)
        iview = corr.view(dtype=np.uint32)
        print (bin(iview[0]))
        iview &= np.uint32(~(255 << 23))
        iview |= (((xi + 127) << 23)).astype(np.uint32)
        print (bin(iview[0]))
        return corr
        #return y*corr

In [ ]:
bin(np.uint32(~(255 << 23)))

In [ ]:
logs = np.linspace(-30., 10., 1000)
y = exp_approx(logs)
ref = np.exp(logs)

In [ ]:
pyplot.plot(logs, np.abs(ref - y) / ref)
pyplot.gca().set(yscale='log')

In [ ]:
pyplot.plot(logs, y)
pyplot.plot(logs, ref)
pyplot.gca().set(yscale='log')

In [ ]:
from path_guiding import ExpApproximation


logs = np.linspace(-30., 10., 1024)
y = np.zeros(len(logs), dtype=np.float32)
for i in range(0, 1024, 16):
    y[i:i+16] = ExpApproximation(logs[i:i+16])
ref = np.exp(logs)
print(y)

pyplot.plot(logs, np.abs(ref - y) / ref)
pyplot.gca().set(yscale='log')
pyplot.show()

pyplot.plot(logs, y)
pyplot.plot(logs, ref)
pyplot.gca().set(yscale='log')
pyplot.show()